This file preserve python code I used for task 2

### Q1: How is the bike share business in San Francisco performing over time?

In [4]:
#import data

from google.cloud import bigquery
Client = bigquery.Client.from_service_account_json(r'key.json')
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=r'key.json'


bqclient = bigquery.Client()

query_string = """
#standardSQL

SELECT 
    trip_id, 
    CAST(start_date AS DATE) AS start_date, 
    FIRST_VALUE(start_station_name) OVER(PARTITION BY start_station_id ORDER BY start_date DESC) AS start_station_name,
    CAST(end_date AS DATE) AS end_date, 
    FIRST_VALUE(end_station_name) OVER(PARTITION BY end_station_id ORDER BY start_date DESC) AS end_station_name,
    TIMESTAMP_DIFF(end_date, start_date, second) AS duration,
    EXTRACT(YEAR from start_date) AS year,
    EXTRACT(TIME from start_date) AS time,
    EXTRACT(DAYOFWEEK from start_date) AS weekday,
    bike_number, 
    subscriber_type, 
    member_birth_year, 
    member_gender 
FROM 
    `bigquery-public-data.san_francisco_bikeshare.bikeshare_trips` y
"""

sf = (
    bqclient.query(query_string)
    .result()
    .to_dataframe(
        create_bqstorage_client=True,
    )
)

In [1]:
# check null
# used subscriber, customer, total
# used bike,  active stations, 
# trips taken by subscriber & customer, duration, by year over date
# heatmap of station

### Q2: What are your observations when you compare bike share users in San Francisco with bike share users in New York?

In [ ]:
# gender & age demo average over day
# comparing temperature effect 
# comparing subscriber & customer weekday & day pattern